In [1]:
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from datetime import datetime

import numpy as np
import pandas as pd
import json
import time


In [2]:
client = Client('   ', '  ')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
info = client.get_all_tickers()

In [5]:
quote_ccy = 'USDT'

In [6]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [7]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [8]:
target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 300 and d['rank'] <= 350]

In [9]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [10]:
target_ticker_list

['FETUSDT',
 'MBLUSDT',
 'STPTUSDT',
 'ANTUSDT',
 'SUNUSDT',
 'UTKUSDT',
 'XVSUSDT',
 'ALPHAUSDT',
 'CTKUSDT',
 'REEFUSDT',
 'ALICEUSDT',
 'PERPUSDT',
 'TLMUSDT',
 'BAKEUSDT',
 'POLSUSDT',
 'XVGUSDT',
 'GHSTUSDT',
 'TRIBEUSDT',
 'DARUSDT',
 'RGTUSDT',
 'MOVRUSDT',
 'ACHUSDT']

In [11]:
# target_ticker_list = ['FIROUSDT']

In [12]:
def is_candidate(filtered_klines_info, tolerable_volume): 
    context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
    is_candidate = False
    
    trigger_kline = filtered_klines_info[-1]

    
    for context_kline in context_klines_info:
        kontext_tol_val = tolerable_volume * 2
        
        volume_gate = float(context_kline[5])  <  kontext_tol_val
        if volume_gate :
            is_candidate = True
        else:
            is_candidate = True
            

    #trigger_gate = float(trigger_kline[5]  > tolerable_volume * 10) and (trigger_kline[-1] < 10 ) and (trigger_kline[-1] > 7 ) 
    
    drop_from_high_gate = trigger_kline[2]/trigger_kline[4] < 1.05
    trigger_gate = trigger_kline[-1] > 10 
    

    if trigger_gate and drop_from_high_gate:
        is_candidate = True
    else:
        is_candidate = False


    return is_candidate

In [13]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[4] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [14]:
trade_list = []
for target_ticker in target_ticker_list:
    
    print('Analyzing Hourly Started ')
    print(target_ticker)

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "30 day ago UTC") 
    length = int(len(klines))
    if(length < 24):
        continue
    
    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)

    for x in range(0, length):
        
        hit_ticker_dictionary = {}
        filtered_klines_info = focused_kline_array[x:x+24]
        if(len(filtered_klines_info) == 10):
            context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
            average_volume = np.mean(context_klines_info[:,5])
            tolerable_volume = average_volume
        
        
            buy_kline_info = filtered_klines_info[-1]


            if(is_candidate(filtered_klines_info, tolerable_volume)):

                hit_ticker_dictionary = {}
                buy_price = buy_kline_info[4]
                hit_ticker_dictionary['ticker'] = target_ticker
                hit_ticker_dictionary['buy_price'] = buy_kline_info[4]

                buy_timesstamp = buy_kline_info[0]
                hit_ticker_dictionary['buy_datetime'] = str(datetime.fromtimestamp(buy_timesstamp // 1000))
                
                proft_percentage = 1.10
                loss_percentage = 0.75
                profit_sell_price = buy_price * proft_percentage
                loss_sell_price = buy_price * loss_percentage
                
                
                usdt = 100
                
                for y in range(x+11, length):
                    high_price = focused_kline_array[y-1:y][0][2]
                    low_price = focused_kline_array[y-1:y][0][3]
                    
                    sell_timesstamp = focused_kline_array[y-1:y][0][0]
                    sell_datetime = str(datetime.fromtimestamp(sell_timesstamp // 1000))


                    if high_price >= profit_sell_price:
                        hit_ticker_dictionary['sell_price'] = profit_sell_price
                        sell_timesstamp = focused_kline_array[y-1:y][0][0]

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = profit_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * proft_percentage - usdt
                        
                        break
                        
                    elif low_price <= loss_sell_price and True:
                        hit_ticker_dictionary['sell_price'] = loss_sell_price
                        sell_timesstamp = focused_kline_array[y-1:y][0][0]

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_kline_array[y-1:y][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = loss_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * loss_percentage - usdt
                        break
                        
                        
                trade_list.append(hit_ticker_dictionary)
                
                busd = 10

                quantity = busd/ buy_kline_info[4]
                quantity_str = "{:0.0{}f}".format(quantity, 2)

                """
                order = client.create_test_order(symbol=target_ticker, side=SIDE_BUY, type='MARKET', quoteOrderQty=busd)
                time.sleep(5)
                order_id = order['orderId'] 
                '''order_id = client.get_all_orders(symbol=target_ticker)[0]['orderId'] '''

                buy_price = order['fills'][0]['price']
                buy_qty = order['executedQty']
                sell_price = buy_price * 1.03



                while client.get_order(symbol=target_ticker, orderId=order_id)['status'] != 'FILLED':
                    time.sleep(10)

                if client.get_order(symbol=target_ticker, orderId=order_id)['status'] == 'FILLED':
                    asset = target_ticker.split("USDT")[0]
                    balance = client.get_asset_balance(asset=asset)
                    free_to_trade = balance['free']
                    sell_price_str = "{:0.0{}f}".format(sell_price, 2)

                    order = client.create_test_order(symbol=target_ticker, side=SIDE_SELL, type='LIMIT', quantity=float(balance['free']), price=sell_price_str, timeInForce=TIME_IN_FORCE_GTC)
                    print('********** SELL ORDER SET ************************')

                    """


Analyzing Hourly Started 
FETUSDT
Analyzing Hourly Started 
MBLUSDT
Analyzing Hourly Started 
STPTUSDT
Analyzing Hourly Started 
ANTUSDT
Analyzing Hourly Started 
SUNUSDT
Analyzing Hourly Started 
UTKUSDT
Analyzing Hourly Started 
XVSUSDT
Analyzing Hourly Started 
ALPHAUSDT
Analyzing Hourly Started 
CTKUSDT
Analyzing Hourly Started 
REEFUSDT
Analyzing Hourly Started 
ALICEUSDT
Analyzing Hourly Started 
PERPUSDT
Analyzing Hourly Started 
TLMUSDT
Analyzing Hourly Started 
BAKEUSDT
Analyzing Hourly Started 
POLSUSDT
Analyzing Hourly Started 
XVGUSDT
Analyzing Hourly Started 
GHSTUSDT
Analyzing Hourly Started 
TRIBEUSDT
Analyzing Hourly Started 
DARUSDT
Analyzing Hourly Started 
RGTUSDT
Analyzing Hourly Started 
MOVRUSDT
Analyzing Hourly Started 
ACHUSDT


In [15]:
trade_list

[]

In [16]:
len(trade_list)

0

In [17]:
df = pd.DataFrame.from_dict(trade_list)

In [18]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] > 0])

KeyError: 'price_diff'

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] < 0])

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df[pd.isna(df['sell_price'])])

In [ ]:
df['proft_loss'].sum()

In [ ]:
with pd.option_context('display.max_rows', None):
    display(df[np.invert(pd.isna(df['sell_price']))])